In [34]:
import pandas as pd
import numpy as np

In [35]:
df=pd.read_csv("adult.csv",encoding='latin-1')
df[df == '?'] = np.nan
for col in ['workclass', 'occupation', 'native.country']:
    df[col].fillna(df[col].mode()[0], inplace=True)
X = df.drop(['income'], axis=1)
y = df['income']

In [36]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from numpy import int64
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
import pickle

In [37]:
scores=[]

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
import pickle

def logc(y_train,y_test,X_train,X_test,index):
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    y_pred = logreg.predict(X_test)
    score=accuracy_score(y_test, y_pred)
    scores.append(score)
    print('Logistic Regression accuracy score with all the features: {0:0.4f}'. format(score))
    filename="logreg"+str(index)+".sav"
    pickle.dump(logreg,open(filename,"wb"))

def xgbs(y_train,y_test,X_train,X_test,index):
    bst = XGBClassifier(n_estimators=6, max_depth=8, learning_rate=0.8, objective='binary:logistic')
    bst.fit(X_train,y_train)
    y_pred = bst.predict(X_test)
    score=accuracy_score(y_test, y_pred)
    scores.append(score)
    print('XGBClassifier accuracy score with all the features: {0:0.4f}'. format(score))
    filename="bst"+str(index)+".sav"
    pickle.dump(bst,open(filename,"wb"))

def rfs(y_train,y_test,X_train,X_test,index):
    rf = RandomForestClassifier(random_state=0,n_estimators=18,max_depth=15)
    rf.fit(X_train,y_train)
    y_pred = rf.predict(X_test)
    score=accuracy_score(y_test, y_pred)
    scores.append(score)
    print('RandomForestClassifier accuracy score with all the features: {0:0.4f}'. format(score))
    filename="rf"+str(index)+".sav"
    pickle.dump(rf,open(filename,"wb"))   
def lgbs(y_train,y_test,X_train,X_test,index):
    clf = lgb.LGBMClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score=accuracy_score(y_test, y_pred)
    scores.append(score)
    print('LGBMClassifier accuracy score with all the features: {0:0.4f}'. format(score))
    filename="clf"+str(index)+".ptl"
    pickle.dump(clf,open(filename,"wb"))

In [39]:
leng=X.shape[0]
X_shap,y_shap=X[:int(leng*0.7)].copy(),y[:int(leng*0.7)].copy()
X_background,y_background=X[int(leng*0.7):].copy(),y[int(leng*0.7):].copy()
categorical = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country']
for feature in categorical:
        le = preprocessing.LabelEncoder()
        X_shap[feature] = le.fit_transform(X_shap[feature])
        X_background[feature] = le.transform(X_background[feature])
scaler = StandardScaler()

X_shap = pd.DataFrame(scaler.fit_transform(X_shap), columns = X.columns)

X_background = pd.DataFrame(scaler.transform(X_background), columns = X.columns)

y_shap[y_shap=="<=50K"]=0
y_shap[y_shap==">50K"]=1
y_background[y_background=="<=50K"]=0
y_background[y_background==">50K"]=1
y_shap=y_shap.astype(int64)
y_background=y_background.astype(int64)


In [40]:
X_shap[0:1].shape

(1, 14)

In [41]:
X_s=X_shap[0:1]
y_s=y_shap[0:1]

In [42]:
import shap
import matplotlib.pyplot as plt
import os
import time
shap_values=None
def load_pre(filename,isTree=True,save=True):
    # print(filename)
    loaded_model = pickle.load(open(filename, 'rb'))
    # y_pred=loaded_model.predict(X_shap)
    # print(accuracy_score(y_shap,y_pred))
    global shap_values

    if isTree:
        explainer = shap.TreeExplainer(loaded_model)
        shap_values = explainer.shap_values(X=X_s)
    else:
        explainer = shap.Explainer(loaded_model,X_background)
        shap_values = explainer.shap_values(X_s)
    # print(shap_values)
    # if isinstance(shap_values,list):
    #     # shap.summary_plot(shap_values[0],X_shap,max_display=10)
    #     # shap.summary_plot(shap_values[1],X_shap,max_display=10)
    #     if save:
    #         np.save("/home/zyxing/adult_income/shaps/"+os.path.basename(filename)+".npy",np.array(shap_values[1]))
    #     else:
    #         shap.summary_plot(shap_values[1],X_shap,max_display=5,show=False)
    #         plt.savefig(str(time.time())+".png")
    # else:
    #     if save:
    #         np.save("/home/zyxing/adult_income/shaps/"+os.path.basename(filename)+".npy",np.array(shap_values))
    #     else:
    #         shap.summary_plot(shap_values,X_shap,max_display=5,show=False)
    #         plt.savefig(str(time.time())+".png")


In [43]:
def findAllFile(base):
    for root, ds, fs in os.walk(base):
        for f in fs:
            fullname = os.path.join(root, f)
            yield fullname

In [44]:
count1=0

for i in [14,8,18,22,26]:#14 8 18 22 26
    load_pre("/home/zyxing/adult_income/models/logreg"+str(i)+".sav",False)

In [45]:
count1=0

for i in findAllFile("/home/zyxing/adult_income/models"):
    name=os.path.basename(i)
    if name.startswith("rf"):
        load_pre(i)
    if name.startswith("logreg"):
        load_pre(i,False)
    elif name.startswith("bst"):
        load_pre(i)
    elif name.startswith("clf"):
        load_pre(i)
    count1+=1
    print(count1)

LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


1
2
3


LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


4
5


LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


...

LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


147
148
149


LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


150
151
152
153
154


LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


In [46]:
import os